In [ ]:
from langchain.agents import tool
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

# get_word_length.invoke("abc")

tools = [get_word_length, TavilySearchResults(max_results=1)]

In [ ]:
from modules.openai_agent import create_agent

model = create_agent(tools=tools, model_only=True, streaming=True)

In [ ]:
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from langchain_core.messages import ToolMessage
import json
from modules.create_graph import create_graph, stream_app


tool_executor = ToolExecutor(tools)

tool_indicator = "tool_calls" #"function_call"

# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if tool_indicator not in last_message.additional_kwargs:
        return "end"
    else:
        return "continue"

# Define the function that calls the model
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}

# Define the function that calls the model
def call_agent(state):
    messages = state['messages']
    response = agent.invoke(
        {"input": messages[0], "chat_history": chat.history})
    return {"messages": [response['output']]}

# Define the function to execute tools
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]
    
    tool = last_message.additional_kwargs[tool_indicator][0]
    action = ToolInvocation(
        tool=tool['function']["name"],
        tool_input=json.loads(tool['function']['arguments'])
    )

    response = tool_executor.invoke(action)
    function_message = ToolMessage(content=str(response), tool_call_id=tool['id'])
    return {"messages": [function_message]}

In [ ]:
nodes = [{"name": "agent", "node": call_model}, {
    "name": "action", "node": call_tool, "condition": should_continue}]

app = create_graph(nodes)

user_input = "what is the weather in sf"
stream_app(app, user_input)